In [ ]:
import requests
import pandas as pd
import networkx as nx
import numpy as np

import pandas as pd
import numpy as np
import spotipy
import spotipy.util as util
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data

import time

In [ ]:
ARTIST = "mumford and sons"

### API keys and main parameters

SetList.fm

In [ ]:
API_KEY = "ad30e075-cc83-41c3-8708-dfb5625a2330"

BASE_URL = "https://api.setlist.fm/rest/1.0"
MIN_SETLIST_LEN = 5
NUM_OF_PAGES = 6

Spotify

In [ ]:
client_id = '1f9afb3cf4f444d3b3a2f9d3bf20a4dc'
client_secret = '12f145a677b54fe6b6b05c52dcbd433b'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

In [ ]:
def get_albums_info(sp_albums):
    album_names = []
    album_uris = []
    for i in range(len(sp_albums['items'])):
        album_names.append(sp_albums['items'][i]['name'])
        album_uris.append(sp_albums['items'][i]['uri'])
    return album_names, album_uris

In [ ]:
def albumSongs(uri, name, spotify_albums):
    album = uri #assign album uri to a_name
    spotify_albums[album] = {} #Creates dictionary for that specific album
    #Create keys-values of empty lists inside nested dictionary for album
    spotify_albums[album]['album'] = [] #create empty list
    spotify_albums[album]['track_number'] = []
    spotify_albums[album]['id'] = []
    spotify_albums[album]['name'] = []
    spotify_albums[album]['uri'] = []
    tracks = sp.album_tracks(album) #pull data on album tracks
    for n in range(len(tracks['items'])): #for each song track
            spotify_albums[album]['album'].append(name) #append album name tracked via album_count
            spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
            spotify_albums[album]['id'].append(tracks['items'][n]['id'])
            spotify_albums[album]['name'].append(tracks['items'][n]['name'])
            spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])
    return spotify_albums

In [ ]:
def get_artist_info(NAME):
    result = sp.search(NAME) #search query
    result['tracks']['items'][0]['artists']

    #Extract Artist's uri
    artist_uri = result['tracks']['items'][0]['artists'][0]['uri']

    #Pull all of the artist's albums
    sp_albums = sp.artist_albums(artist_uri, album_type='album')
    album_names, album_uris = get_albums_info(sp_albums)

    spotify_albums = {}
    album_count = 0
    checked_albs = []
    for i, name in zip(album_uris, album_names): #each album
        if name not in checked_albs:
            checked_albs.append(name)
            spotify_albums = albumSongs(i, name, spotify_albums)
            album_count+=1 #Updates album count once all tracks have been addedb
  
    return spotify_albums

In [ ]:
def audio_features(album):
    #Add new key-values to store audio features
    spotify_albums[album]['popularity'] = []
    #create a track counter
    track_count = 0
    for track in spotify_albums[album]['uri']:
        #pull audio features per track
        features = sp.audio_features(track)
        pop = sp.track(track)
        spotify_albums[album]['popularity'].append(pop['popularity'])
        track_count+=1

In [ ]:
def info_pd(spotify_albums):
    sleep_min = 2
    sleep_max = 3
    start_time = time.time()
    request_count = 0


    for i in spotify_albums:
        audio_features(i)
        request_count+=1
        if request_count % 5 == 0:
            print(str(request_count) + " playlists completed")
            time.sleep(np.random.uniform(sleep_min, sleep_max))
    dic_df = {}
    dic_df['album'] = []
    dic_df['track_number'] = []
    dic_df['id'] = []
    dic_df['name'] = []
    dic_df['uri'] = []
    dic_df['popularity'] = []
    for album in spotify_albums: 
        for feature in dic_df.keys():
            dic_df[feature].extend(spotify_albums[album][feature])

    return pd.DataFrame.from_dict(dic_df)

In [ ]:
def get_setlist_songs(ARTIST, BASE_URL, API_KEY):
    artist_url = "{}/search/artists/?sort=relevance&artistName={}".format(BASE_URL, ARTIST)
    headers = {'x-api-key': API_KEY, 'Accept': 'application/json'}
    r = requests.get(artist_url, headers=headers)
    artists = r.json().get('artist')
    artist = artists[0]
    
    responses = []
    for i in range(1, NUM_OF_PAGES): 
        playlist_url = "{}/artist/{}/setlists/?p={}".format(BASE_URL, artist.get('mbid'), i)
        r = requests.get(playlist_url, headers=headers)
        responses.append(r.json())
        
    responses_setlist = map(lambda x: x.get('setlist'), responses)
    flattened_setlists = [item for sublist in responses_setlist for item in sublist]
    cleaned_setlists = list(map(lambda x: {
        'eventDate': x.get('eventDate'), 
        'songs': list(map(lambda z: z.get('name'), [item for sublist in map(lambda y: y.get('song'), x.get('sets').get('set')) for item in sublist])), 
        'tour': x.get('tour').get('name'),
        'country': x.get('venue').get('city').get('country').get('code')
        }, flattened_setlists))
    only_songs = list(map(lambda x: ["begin"] + x.get('songs') + ["end"], cleaned_setlists))
    
    return only_songs

In [ ]:
def song_list_to_df(ARTIST, BASE_URL, API_KEY):
    songs = get_setlist_songs(ARTIST, BASE_URL, API_KEY)
    pairs = []
    for setlist in songs:
        for i in range(0, len(setlist) - 1):
            pairs.append((setlist[i], setlist[i+1]))

    df_pairs = pd.DataFrame(pairs)

    df_pairs = df_pairs.rename(columns={0:'song_org', 1:'next_song'})
    df_pairs = df_pairs.groupby(['song_org', 'next_song']).size().reset_index().rename(columns={0:'weight'})
    
    return df_pairs

In [ ]:
df_pairs = song_list_to_df(ARTIST, BASE_URL, API_KEY)

In [ ]:
df_pairs.head()

In [ ]:
spotify_albums = get_artist_info(ARTIST)
pd_artist = info_pd(spotify_albums)

In [ ]:
pd_artist[(pd_artist.album == 'Delta') & (pd_artist.name == 'Delta')]